In [ ]:
class resnet_feature_extractor(torch.nn.Module):
    def __init__(self):
        """This class extracts the feature maps from a pretrained Resnet model."""
        super(resnet_feature_extractor, self).__init__()
        self.model = resnet50(weights=ResNet50_Weights.DEFAULT)


        self.model.eval()
        for param in self.model.parameters():
            param.requires_grad = False

        # Hook to extract feature maps
        def hook(module, input, output) -> None:
            """This hook saves the extracted feature map on self.featured."""
            self.features.append(output)

        self.model.layer2[-1].register_forward_hook(hook)
        self.model.layer3[-1].register_forward_hook(hook)

    def forward(self, input):

        self.features = []
        with torch.no_grad():
            _ = self.model(input)

        self.avg = torch.nn.AvgPool2d(3, stride=1)
        fmap_size = self.features[0].shape[-2]         # Feature map sizes h, w
        self.resize = torch.nn.AdaptiveAvgPool2d(fmap_size)

        resized_maps = [self.resize(self.avg(fmap)) for fmap in self.features]
        patch = torch.cat(resized_maps, 1)            # Merge the resized feature maps
        patch = patch.reshape(patch.shape[1], -1).T   # Craete a column tensor

        return patch


In [ ]:

image = Image.open(r'/content/drive/MyDrive/DatasetBlack1/testblack/tr1/trplastic_IMG_0002_40.jpg')
image = transform(image).unsqueeze(0).cuda()
backbone = resnet_feature_extractor().cuda()
feature = backbone(image)
print(backbone.features[0].shape)
print(backbone.features[1].shape)
#print(backbone.features[2].shape)
print(feature.shape)

plt.imshow(image[0].permute(1, 2, 0).cpu())

In [ ]:
memory_banktr =[]

folder_path = Path(r'/content/drive/MyDrive/DatasetBlack1/tr1/train/good')

for pth in tqdm(folder_path.iterdir(),leave=False):

    with torch.no_grad():
        data = transform(Image.open(pth)).cuda().unsqueeze(0)
        features = backbone(data)

        memory_banktr.append(features.cpu().detach())

memory_banktr = torch.cat(memory_banktr,dim=0).cuda()

In [ ]:
print(f"Shape of memory_bank1: {memory_banktr.shape}")
print(f"Data type of memory_bank1: {memory_banktr.dtype}")

In [ ]:
memory_bank11 = memory_banktr.clone()
selected_indices = np.random.choice(len(memory_bank11), size=len(memory_bank11)//1, replace=False)
memory_bank11 = memory_bank11[selected_indices]

In [ ]:
memory_bank11 = torch.load("/content/drive/MyDrive/memory_bankblv1.pt").cuda()

In [ ]:
y_score1=[]
folder_path = Path(r'/content/drive/MyDrive/DatasetBlack1/tr1/train/good')

for pth in tqdm(folder_path.iterdir(),leave=False):
    data1 = transform(Image.open(pth)).cuda().unsqueeze(0)
    with torch.no_grad():
        features1 = backbone(data1)
    distances1 = torch.cdist(features1, memory_bank11, p=2.0)
    dist_score1, dist_score_idxs1 = torch.min(distances1, dim=1)
    s_star1 = torch.max(dist_score1)
    segm_map1 = dist_score1.view(1, 1, 28, 28)

    y_score1.append(s_star1.cpu().numpy())

In [ ]:
best_threshold1 = np.mean(y_score1) + 2 * np.std(y_score1)
print(best_threshold1)
plt.hist(y_score1,bins=50)
plt.vlines(x=best_threshold1,ymin=0,ymax=30,color='r')
plt.show()

In [ ]:
y_score1 = []
y_true1=[]

for classes in ['good','bad']:
    folder_path = Path(r'/content/drive/MyDrive/DatasetBlack1/tr1/test/{}'.format(classes))

    for pth in tqdm(folder_path.iterdir(),leave=False):

        class_label = pth.parts[-2]
        with torch.no_grad():
            test_image1 = transform(Image.open(pth)).cuda().unsqueeze(0)
            features1 = backbone(test_image1)

        distances1 = torch.cdist(features1, memory_bank11, p=2.0)
        dist_score1, dist_score_idxs = torch.min(distances1, dim=1)
        s_star1 = torch.max(dist_score1)
        segm_map1 = dist_score1.view(1, 1, 28, 28)

        y_score1.append(s_star1.cpu().numpy())
        y_true1.append(0 if class_label == 'good' else 1)

In [ ]:
# plotting the y_score values which do not belong to 'good' class
y_score_nok1 = [score  for score,true in zip(y_score1,y_true1) if true==1]
plt.hist(y_score_nok1,bins=50)
plt.vlines(x=best_threshold1,ymin=0,ymax=30,color='r')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay, f1_score
# Calculate AUC-ROC score
auc_roc_score1 = roc_auc_score(y_true1, y_score1)
print("AUC-ROC Score:", auc_roc_score1)
# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_true1, y_score1)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc_roc_score1)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()
f1_scores = [f1_score(y_true1, y_score1 >= threshold) for threshold in thresholds]
# Select the best threshold based on F1 score
best_threshold = thresholds[np.argmax(f1_scores)]

print(f'best_threshold = {best_threshold}')

# Generate confusion matrix
cm = confusion_matrix(y_true1, (y_score1 >= best_threshold).astype(int))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['OK','NOK'])
disp.plot()
plt.show()

In [ ]:
# begin = time.time()
# Đảm bảo backbone ở chế độ đánh giá
backbone.eval()

# Đường dẫn đến thư mục cần kiểm tra
test_path = Path('/content/drive/MyDrive/DatasetBlack1/testblack/tr1')
best_threshold2 = 12.82919960
# Lặp qua tất cả các tệp .bmp trong thư mục
for path in test_path.glob('*.jpg'):  # Tìm các tệp .bmp trong thư mục 'bad'
    test_image = transform(Image.open(path)).cuda().unsqueeze(0)

    # Dự đoán với mô hình
    with torch.no_grad():
        features = backbone(test_image)

    # Tính toán khoảng cách
    distances = torch.cdist(features, memory_bank11, p=2.0)
    dist_score, dist_score_idxs = torch.min(distances, dim=1)
    s_star = torch.max(dist_score)

    # Tạo segmentation map
    segm_map = dist_score.view(1, 1, 28, 28)
    segm_map = torch.nn.functional.interpolate(
        segm_map,
        size=(224, 224),
        mode='bilinear'
    ).cpu().squeeze().numpy()
    # Tính điểm bất thường
    y_score_image = s_star.cpu().numpy()
    
    y_pred_image = 1 * (y_score_image >= best_threshold)
    class_label = ['GOOD', 'BAD']
    y_score_image = s_star.cpu().numpy()
    #best_threshold3 = best_threshold1-10

    # In kết quả
    print(f'File name: {path.name}')
    print(f'Anomaly score: {y_score_image:0.8f}')
    print(f'Prediction: {class_label[y_pred_image]}')

    # Hiển thị kết quả
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(test_image.squeeze().permute(1, 2, 0).cpu().numpy())
    plt.title(f'File: {path.name}')

    plt.subplot(1, 3, 2)
    heat_map = segm_map
    plt.imshow(heat_map, cmap='viridis', vmin=best_threshold, vmax=best_threshold * 1.5)
    plt.title(f'Defective score: {y_score_image / best_threshold:0.4f} || {class_label[y_pred_image]}')

    plt.show()

    # time.sleep(0.1)
    #clear_output(wait=True)
# end = time.time()
# print(end-begin)

In [ ]:
torch.save(memory_bank11, "/content/drive/MyDrive/memory_banktlv1.pt")